This notebook showcases `MosaicWidget`'s interoperability with various dataframe libraries. Leveraging [Narwhals](https://github.com/narwhals-dev/narwhals) as an abstraction layer, `MosaicWidget` accepts data from backends like Pandas, Polars, Dask, Modin, PyArrow tables, and DuckDB relations without requiring manual conversion.

In [1]:
# Install dataframe libs not part of Mosaic dependencies
!uv pip install dask ibis-framework pyarrow-hotfix modin polars -q

In [2]:
import dask.dataframe as dd
import duckdb
import ibis
import modin.pandas as md
import pandas as pd
import polars as pl
import pyarrow as pa

import yaml
from mosaic_widget import MosaicWidget

from IPython.display import display, Markdown

In [3]:
# Mandatory Modin configuration
import os

os.environ["MODIN_ENGINE"] = "python"
os.environ["MODIN_STORAGE_FORMAT"] = "pandas"

In [4]:
# Optional: enable logging in Mosaic widget to get informed about dataset conversion details
import logging

logging.basicConfig(level=logging.WARN)

In [5]:
# Load weather spec, remove data key to ensure load from demonstrated backends
with open("../../specs/yaml/weather.yaml") as f:
    spec = yaml.safe_load(f)
    spec.pop("data")

In [6]:
csv_path = "../../data/seattle-weather.csv"

weather_dd = dd.read_csv(csv_path, parse_dates=["date"])
weather_duckdb = duckdb.query(f"select * from '{csv_path}'")
weather_ibis = ibis.read_csv(csv_path)
weather_modin = md.read_csv(csv_path, parse_dates=["date"])
weather_pd = pd.read_csv(csv_path, parse_dates=["date"])
weather_pa = pa.Table.from_pandas(weather_pd)
weather_pl = pl.read_csv(csv_path, try_parse_dates=True)

datasets = [
    # Native
    weather_pd,
    weather_pl,
    weather_pa,
    # Eager
    weather_modin,
    # Lazy
    weather_dd,
    weather_ibis,
    weather_duckdb,
]

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


In [7]:
for dataset in datasets:
    backend_name = str(type(dataset))[8:-2]
    display(Markdown(f"### Mosaic Widget with data from `{backend_name}` backend"))
    display(MosaicWidget(spec, data={"weather": dataset}))

### Mosaic Widget with data from `pandas.core.frame.DataFrame` backend

MosaicWidget(spec={'meta': {'title': 'Seattle Weather', 'description': "An interactive view of Seattle's weath…

### Mosaic Widget with data from `polars.dataframe.frame.DataFrame` backend

MosaicWidget(spec={'meta': {'title': 'Seattle Weather', 'description': "An interactive view of Seattle's weath…

### Mosaic Widget with data from `pyarrow.lib.Table` backend

...

MosaicWidget(spec={'meta': {'title': 'Seattle Weather', 'description': "An interactive view of Seattle's weath…

### Mosaic Widget with data from `ibis.expr.types.relations.Table` backend

MosaicWidget(spec={'meta': {'title': 'Seattle Weather', 'description': "An interactive view of Seattle's weath…

### Mosaic Widget with data from `duckdb.duckdb.DuckDBPyRelation` backend

MosaicWidget(spec={'meta': {'title': 'Seattle Weather', 'description': "An interactive view of Seattle's weath…